<a href="https://colab.research.google.com/github/YIF27/Dataset-of-Spinel/blob/main/bond_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pymatgen.core import Structure
class BondAnalyzer:
def __init__(self, input_data):
        if isinstance(input_data, Structure):
            self.structure = input_data
        elif isinstance(input_data, str):  # Assuming a file path is given as a string
            self.structure = Structure.from_file(input_data)
        else:
            raise ValueError("Input must be a pymatgen Structure or a path to a POSCAR file.")


    @staticmethod
    def generate_bond_info(element1, element2):
        # Your function code goes here
        # ...

    # You can also add more related functions or methods in the future.

# Usage in the same file:
analyzer = BondAnalyzer()
result = analyzer.generate_bond_info("H", "O")

# If you want to use this class in another file, you can import it.
# Assuming you saved this class in a file named "bond_analyzer.py":
# from bond_analyzer import BondAnalyzer
